# Анализ данных сервиса для чтения книг по подписке

 Анализируется база данных сервиса для чтения книг по подписке. На основе исследования будет сформулировано  ценностное предложение для нового продукта. 
 
**Задачи исследования** 
- Определить количество книг, вышедших после 1 января 2000 года
- Посчитать количество обзоров и среднюю оценку для каждой книги
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц
- Определить автора с самой высокой средней оценкой книг (учитываются только книги с 50 и более оценками)
- Определить среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Общая информация

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
# функция выполнения SQL запроса
def select_data(query):
    return pd.io.sql.read_sql(query, con = engine)

In [3]:
# цикл для получения первых строк каждой таблицы 
tables = ('books', 'authors','publishers','ratings','reviews')
for table in tables:
    query = '''SELECT * FROM {} LIMIT 5'''.format(table)
    display(select_data(query))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Количество книг, вышедших после 1 января 2000 года

In [4]:
query = '''
SELECT COUNT(book_id) AS qn_book
FROM books 
WHERE publication_date > '2000-01-01'
'''

In [5]:
select_data(query)

,qn_book
0,819


Количество книг, вышедших после 1 января 2000 года:819 книг.


## Количество обзоров и средняя оценка для каждой книги

In [6]:
query = '''
SELECT b.book_id,
b.title,
r.qn_review,
ra.avg_rating
FROM books b
LEFT JOIN (SELECT book_id,
                    COUNT (review_id) as qn_review
                    FROM reviews
                    GROUP BY book_id) as r ON b.book_id = r.book_id
LEFT JOIN (SELECT book_id,
                    ROUND(AVG(rating),2) as avg_rating
             FROM ratings
             GROUP BY book_id) as ra ON b.book_id = ra.book_id
ORDER BY ra.avg_rating  DESC 
'''

In [7]:
select_data(query)

,book_id,title,qn_review,avg_rating
0,518,Pop Goes the Weasel (Alex Cross #5),2.0,5.00
1,732,The Ghost Map: The Story of London's Most Terr...,2.0,5.00
2,347,In the Hand of the Goddess (Song of the Liones...,2.0,5.00
3,610,Tai-Pan (Asian Saga #2),2.0,5.00
4,330,How to Be a Domestic Goddess: Baking and the A...,1.0,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3.0,2.25
996,371,Junky,2.0,2.00
997,316,His Excellency: George Washington,2.0,2.00
998,202,Drowning Ruth,3.0,2.00


Диапозон количества обзоров: от 1120 до 2.  
Максимальная оценка у книги: 5.0; минимальная: 1.5. 
Стоит отметить, что в таблице с обзорами, есть обзоры на 994 книги из 1000.

## Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [8]:
query = '''
SELECT b.publisher_id,
p.publisher,
COUNT(b.book_id) as book_qn
FROM books b 
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE num_pages>50
GROUP BY b.publisher_id, p.publisher
ORDER BY book_qn DESC 
LIMIT 1
'''

In [9]:
select_data(query) 

,publisher_id,publisher,book_qn
0,212,Penguin Books,42


Издательство Penguin Books выпустило больше всего книг толщиной больше 50 страниц

## Автор с самой высокой средней оценкой книг 

In [10]:
query = '''
SELECT b.author_id,
author,
AVG(q.avg_rating)
FROM books b JOIN 
(SELECT book_id, AVG(rating) as avg_rating
FROM ratings
GROUP BY book_id
HAVING  COUNT(rating)>50) q ON b.book_id = q.book_id
JOIN authors a ON b.author_id = a.author_id
GROUP BY b.author_id, author
ORDER BY avg DESC
LIMIT 1
'''

In [11]:
select_data(query) 

,author_id,author,avg
0,236,J.K. Rowling/Mary GrandPré,4.283844


Автор с самой высокой средней оценкой книг: J.K. Rowling

## Среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [12]:
query = '''
SELECT AVG (qn_review)::int as avg_qn
FROM (SELECT username,
COUNT(review_id) as qn_review
FROM reviews
GROUP BY username) q
WHERE username in (SELECT username
FROM ratings 
GROUP BY username
HAVING COUNT(rating)>50)

'''

In [13]:
select_data(query) 

,avg_qn
0,24


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок: 24

**Вывод**

В базе данных: 5 таблиц с информацией о книгах, авторах, издательствах, а также пользовательские обзоры книг. 
- books            
- authors
- publishers
- ratings
- reviews

Количество книг, вышедших после 1 января 2000 года:819 книг.
Диапозон количества обзоров: от 1120 до 2.  
Максимальная оценка у книги: 5.0; минимальная: 1.5. 
Издательство Penguin Books выпустило больше всего книг толщиной больше 50 страниц
Автор с самой высокой средней оценкой книг: J.K. Rowling